# Geographical Visualization SuperStore

## 1. Import files and libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

In [3]:
# Import ".json" file for the U.S. 

country_geo = r'C:/Users/Carolina/Documents/CareerFoundry/Data Immersion/Achievement 6- Advanced Analytics & Dashboard Design/02-SS Data/Raw Data/us-states.json'

In [4]:
# path into a string 
path= r'C:\Users\Carolina\Documents\CareerFoundry\Data Immersion\Achievement 6- Advanced Analytics & Dashboard Design'

In [5]:
# Import dataset into Jupyter as df_st  
df_ss = pd.read_csv(os.path.join(path, '02-SS Data','Prepared Data','Superstore3.csv'))

In [6]:
df_ss.describe()

Discount       Profit     Quantity         Sales
count  9993.000000  9993.000000  9993.000000   9993.000000
mean      0.156188    28.660971     3.789753    229.852846
std       0.206457   234.271476     2.225149    623.276074
min       0.000000 -6599.978000     1.000000      0.444000
25%       0.000000     1.731000     2.000000     17.280000
50%       0.200000     8.671000     3.000000     54.480000
75%       0.200000    29.364000     5.000000    209.940000
max       0.800000  8399.976000    14.000000  22638.480000

## 2. Data Wrangling and Geographical Visualization per Category 

Let us remember that the variable purchase type describe three types of purchases: big, medium and small. The first group consist of purchases less than $\$$55, the second between $\$$55 and $\$$230 and the last one above $\$$230. So before doing any geographical visualizations, Let us explore how many points each of these partitions have.  

In [7]:
df_ss['type purchase'].value_counts()

Low purchase       4996
Medium purchase    2639
Big purchase       2358
Name: type purchase, dtype: int64

In [8]:
df_ss['Sales'].max()

22638.48

Big Purchases ranges from $\$$230 until $\$$22,638.48 which is a very extensive range and will not help to extract any valuable insights when doing the visualizations. Therefore, let us rather explore the data by categories and types of purchases to see if there are any outliers. 

In [9]:
# Group by category and type of purchase to see in % how much each contribute to profit
df_ss.groupby(['Category','type purchase']).agg({'Profit': ['sum','count']}).apply(lambda x: 100 * x / float(x.sum()))

Profit           
                                       sum      count
Category        type purchase                        
Furniture       Big purchase      5.562744   9.096367
                Low purchase      0.750189   5.724007
                Medium purchase   0.133557   6.394476
Office Supplies Big purchase     26.528977   6.614630
                Low purchase      5.399665  40.498349
                Medium purchase  10.839136  13.189232
Technology      Big purchase     45.431127   7.885520
                Low purchase      0.522126   3.772641
                Medium purchase   4.832480   6.824777

A quick scan let us see that: <br> 
-45% of the profit is concentrated in big purchases from Technology.This is followed by 26% in big purchases from office supplies. <br> 
-40% of the orders which are made in low purchases from office supplies only contribute to 5% of the profit.<br><br>
Let's look at each category in more detail. 

### 2.1 Category of Tecnnology 

In [10]:
#Create a subset with the Technology category
df_tecno = df_ss[df_ss['Category']=='Technology']

In [11]:
# To discard outliers let us group the technology category by type of purchase and subcategory. 
df_tecno.groupby(['Sub-Category','type purchase']).agg({'Sales': ['sum','mean', 'min', 'max', 'count', 'median']})

Sales                                   \
                                     sum         mean      min        max   
Sub-Category type purchase                                                  
Accessories  Big purchase     120354.548   581.422937  234.450   3347.370   
             Low purchase       6885.220    30.737589    0.990     54.384   
             Medium purchase   40140.550   116.687645   55.176    227.976   
Copiers      Big purchase     149528.030  2198.941618  299.990  17499.950   
Machines     Big purchase     187091.411  2011.735602  239.976  22638.480   
             Low purchase        169.840    28.306667   11.560     52.440   
             Medium purchase    1977.380   123.586250   59.994    224.937   
Phones       Big purchase     284533.488   677.460686  230.376   4548.810   
             Low purchase       4387.672    29.848109    2.970     54.368   
             Medium purchase   41085.894   127.595944   55.176    227.460   

                                              
                             count    median  
Sub-Category type purchase                    
Accessories  Big purchase      207   399.960  
             Low purchase      224    30.460  
             Medium purchase   344   102.185  
Copiers      Big purchase       68  1099.980  
Machines     Big purchase       93   821.300  
             Low purchase        6    30.135  
             Medium purchase    16    98.994  
Phones       Big purchase      420   497.075  
             Low purchase      147    31.968  
             Medium purchase   322   119.950

In [12]:
# Let us view the results by percentage
df_tecno.groupby(['Sub-Category','type purchase']).agg({'Profit': ['sum','count']}).apply(lambda x: 100 * x / float(x.sum()))


Profit           
                                    sum      count
Sub-Category type purchase                        
Accessories  Big purchase     21.585112  11.207363
             Low purchase      0.811270  12.127775
             Medium purchase   6.434975  18.624797
Copiers      Big purchase     38.237149   3.681646
Machines     Big purchase      2.697623   5.035192
             Low purchase     -0.026372   0.324851
             Medium purchase  -0.344237   0.866270
Phones       Big purchase     26.936593  22.739578
             Low purchase      0.243197   7.958852
             Medium purchase   3.424690  17.433676

Overall, copiers account for 38% of the profit with only big purchases that represent 3.7% of the total purchases in technology cateogry. This is followed by big purchases from phones and accesories which make approximately 49% of the profit (aprox. 27% + 22% =49%) and account for 34% of the total purchases. With a low contribution, machines and low and medium purchases of phones and accesories contribute to only 13% of the profit and account for approximately 62% of the purchases.  

Therefore we can say there are no outliers.<br> 

Besides this, we concluded from the prevoius exercise that the technology category is the one that most contributes to profitablity. Therefore let us explore geographically where these profitable purchases are located. 


In [13]:
df_copiers= df_tecno[df_tecno['Sub-Category']=='Copiers']

In [14]:
df_copiers.shape

(68, 19)

In [15]:
# Create a data frame with just the states and the values for rating we want plot for the copiers subcategory
data_map1 = df_copiers[['State','Profit']]
data_map1.describe()

Profit
count    68.000000
mean    817.909190
std    1460.921156
min      59.998000
25%     164.245725
50%     332.994200
75%     848.736375
max    8399.976000

In [16]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_map1,
    columns = ['State','Profit'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'RdPu', fill_opacity=0.7, line_opacity=0.1,
    legend_name = "Profit").add_to(map)
folium.LayerControl().add_to(map)

map

In [17]:
df_copiers.groupby('State').agg({'Profit': ['mean']}).sort_values(('Profit', 'mean', ),ascending=False)

Profit
                       mean
State                      
Delaware        5039.985600
Indiana         4424.983500
Washington      1888.484840
Rhode Island    1745.969400
Georgia         1439.976000
Montana         1379.977000
Missouri        1268.481600
New York         834.278571
Virginia         751.962400
Kentucky         629.991000
Massachusetts    467.992200
California       464.108935
Illinois         457.991650
Utah             449.985000
Pennsylvania     347.196580
Texas            325.992300
Alabama          314.989500
Minnesota        274.995000
Michigan         242.494000
Florida          200.995900
Colorado         164.997000
North Carolina   119.996000
Ohio             111.748075

#### What can we answer so far from previous Questions?
-From the big purchases of technology, which products most contributed to profit?
<br>- Where are the big profitable purchases concentrated? And among which and how many customers?

<br>For technology, the most profitable sales are from copiers which represent 45% of the profit and 8% of the total amount of orders. The regions were most of these sales happen is Indiana, Delaware and Washington. 

### Category of Office Supplies

In [18]:
#Create a subset with the Office Supplies category
df_office = df_ss[df_ss['Category']=='Office Supplies']

In [19]:
df_office.shape

(6026, 19)

In [20]:
# To discard outliers let us group the technology category by type of purchase and subcategory. 
df_office.groupby(['Sub-Category','type purchase']).agg({'Sales': ['sum','mean', 'min', 'max', 'count', 'median']})

Sales                                  \
                                     sum         mean      min       max   
Sub-Category type purchase                                                 
Appliances   Big purchase      82433.072   686.942267  235.950  2625.120   
             Low purchase       4153.143    25.170564    0.444    54.320   
             Medium purchase   20945.946   115.723459   54.480   227.840   
Art          Big purchase       3694.084   369.408400  255.850  1113.024   
             Low purchase      10762.230    16.281740    1.344    54.336   
             Medium purchase   12662.478   101.299824   54.660   227.880   
Binders      Big purchase     159254.045  1234.527481  232.400  9892.740   
             Low purchase      19418.043    16.841321    0.556    53.984   
             Medium purchase   24740.645   102.658278   54.792   222.352   
Envelopes    Big purchase       4862.780   324.185333  247.840   604.656   
             Low purchase       3357.714    20.599472    1.632    52.512   
             Medium purchase    8255.908   108.630368   54.900   223.888   
Fasteners    Low purchase       2817.448    13.165645    1.240    52.290   
             Medium purchase     206.832    68.944000   55.104    93.360   
Labels       Big purchase       3631.828   453.978500  235.944   786.480   
             Low purchase       5139.820    16.265253    2.088    51.750   
             Medium purchase    3714.664    92.866600   55.440   196.620   
Paper        Big purchase      19577.172   337.537448  239.500   733.950   
             Low purchase      21811.084    22.098363    3.380    53.952   
             Medium purchase   37090.950   114.126000   54.816   229.544   
Storage      Big purchase     181249.602   614.405431  230.376  2934.330   
             Low purchase       7644.018    30.576072    4.464    54.320   
             Medium purchase   34949.988   116.112917   54.500   229.940   
Supplies     Big purchase      41400.212  1592.315846  231.720  8187.650   
             Low purchase       2918.624    21.149449    1.744    52.590   
             Medium purchase    2354.702    90.565462   54.900   190.896   

                                             
                             count   median  
Sub-Category type purchase                   
Appliances   Big purchase      120  465.160  
             Low purchase      165   23.992  
             Medium purchase   181   98.112  
Art          Big purchase       10  276.325  
             Low purchase      661   12.672  
             Medium purchase   125   87.920  
Binders      Big purchase      129  679.960  
             Low purchase     1153   12.672  
             Medium purchase   241   91.200  
Envelopes    Big purchase       15  304.900  
             Low purchase      163   16.980  
             Medium purchase    76   97.206  
Fasteners    Low purchase      214   10.499  
             Medium purchase     3   58.368  
Labels       Big purchase        8  459.485  
             Low purchase      316   14.400  
             Medium purchase    40   87.710  
Paper        Big purchase       58  313.256  
             Low purchase      987   19.440  
             Medium purchase   325   99.136  
Storage      Big purchase      295  484.650  
             Low purchase      250   31.440  
             Medium purchase   301   99.488  
Supplies     Big purchase       26  622.774  
             Low purchase      138   17.672  
             Medium purchase    26   69.500

In [21]:
# Let us view the results by percentage
df_office.groupby(['Sub-Category']).agg({'Profit': ['sum','count']}).apply(lambda x: 100 * x / float(x.sum()))

Profit           
                    sum      count
Sub-Category                      
Appliances    14.807647   7.733156
Art            5.329206  13.209426
Binders       24.672680  25.273813
Envelopes      5.685469   4.215068
Fasteners      0.775175   3.601062
Labels         4.527894   6.040491
Paper         27.800920  22.734816
Storage       17.371775  14.039164
Supplies      -0.970766   3.153004

For offfice supplies 4 sub-categories(paper, binders, storage and appliances) hold 85% of the profitability of this category. Overall, this category contributes to approximately 41% of the total profit. Let's explore these subcagories by type of purchase.

In [22]:
#Let us create a subset with the 4 subcategories that are most profitable
value_list=['Paper', 'Binders', 'Storage', 'Appliances']
boolean_series = df_office['Sub-Category'].isin(value_list)
filtered_df = df_office[boolean_series]

In [23]:
#Let us check if the subset was created correctly by checking the subcategories
filtered_df['Sub-Category'].value_counts()

Binders       1523
Paper         1370
Storage        846
Appliances     466
Name: Sub-Category, dtype: int64

In [24]:
filtered_df.head()

Category             City        Country    Customer Name  \
4   Office Supplies  Fort Lauderdale  United States   Sean O'Donnell   
8   Office Supplies      Los Angeles  United States  Brosina Hoffman   
9   Office Supplies      Los Angeles  United States  Brosina Hoffman   
12  Office Supplies          Concord  United States     Andrew Allen   
13  Office Supplies          Seattle  United States     Irene Maddox   

    Discount  Order Date        Order ID Manufacturer  \
4        0.2  2016-10-11  US-2016-108966        Eldon   
8        0.2  2015-06-09  CA-2015-115812          DXL   
9        0.0  2015-06-09  CA-2015-115812       Belkin   
12       0.2  2018-04-15  CA-2018-114412        Xerox   
13       0.2  2017-12-05  CA-2017-161389     Fellowes   

                                         Product Name    Profit  Quantity  \
4                      Eldon Fold 'N Roll Cart System    2.5164         2   
8   DXL Angle-View Binders with Locking Rings by S...    5.7825         3   
9                    Belkin F5C206VTEL 6 Outlet Surge   34.4700         5   
12                                         Xerox 1967    5.4432         3   
13        Fellowes PB200 Plastic Comb Binding Machine  132.5922         3   

   Region    Sales   Segment   Ship Date       Ship Mode           State  \
4   South   22.368  Consumer  2016-10-18  Standard Class         Florida   
8    West   18.504  Consumer  2015-06-14  Standard Class      California   
9    West  114.900  Consumer  2015-06-14  Standard Class      California   
12  South   15.552  Consumer  2018-04-20  Standard Class  North Carolina   
13   West  407.976  Consumer  2017-12-10  Standard Class      Washington   

   Sub-Category    type purchase  
4       Storage     Low purchase  
8       Binders     Low purchase  
9    Appliances  Medium purchase  
12        Paper     Low purchase  
13      Binders     Big purchase

In [25]:
# Let us view the results by percentage
filtered_df.groupby(['Sub-Category', 'type purchase']).agg({'Profit': ['sum', 'count']}).apply(lambda x: 100 * x / float(x.sum()))

Profit           
                                    sum      count
Sub-Category type purchase                        
Appliances   Big purchase     18.667841   2.853746
             Low purchase     -1.294574   3.923900
             Medium purchase   0.118899   4.304400
Binders      Big purchase     26.197523   3.067776
             Low purchase     -1.159253  27.419738
             Medium purchase   4.107387   5.731272
Paper        Big purchase      8.300845   1.379310
             Low purchase      9.019938  23.472057
             Medium purchase  15.520242   7.728894
Storage      Big purchase     17.519520   7.015458
             Low purchase      0.832820   5.945303
             Medium purchase   2.168812   7.158145

Except for all the type of purchases in the paper subcategory, only the big purchases in the remaining subcategories are the most profitable and these ones are the ones we are interested in visualizing geographically. Therefore, we will only plot profit ratio by subcategory and type of purchase that are bigger than 8% (refer to the grouping table above).   

Let us create a new column that contains the profit ratio per subcategory and type of purchase. 

In [26]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Appliances') & (filtered_df['type purchase'] == 'Big purchase'), 'profit ratio'] = 18.667841

C:\Users\Carolina\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Carolina\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [27]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Appliances') & (filtered_df['type purchase'] == 'Medium purchase'), 'profit ratio'] = -1.294574

In [28]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Appliances') & (filtered_df['type purchase'] == 'Low purchase'), 'profit ratio'] = 0.118899

In [29]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Binders') & (filtered_df['type purchase'] == 'Big purchase'), 'profit ratio'] = 26.197523

In [30]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Binders') & (filtered_df['type purchase'] == 'Medium purchase'), 'profit ratio'] = 4.107387

In [31]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Binders') & (filtered_df['type purchase'] == 'Low purchase'), 'profit ratio'] = -1.159253

In [32]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Paper') & (filtered_df['type purchase'] == 'Big purchase'), 'profit ratio'] = 8.300845

In [33]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Paper') & (filtered_df['type purchase'] == 'Medium purchase'), 'profit ratio'] = 15.520242

In [34]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Paper') & (filtered_df['type purchase'] == 'Low purchase'), 'profit ratio'] = 9.019938

In [35]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Storage') & (filtered_df['type purchase'] == 'Big purchase'), 'profit ratio'] = 17.519520 

In [36]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Storage') & (filtered_df['type purchase'] == 'Medium purchase'), 'profit ratio'] = 2.168812

In [37]:
filtered_df.loc[(filtered_df['Sub-Category'] == 'Storage') & (filtered_df['type purchase'] == 'Low purchase'), 'profit ratio'] = 0.832820

In [38]:
# let us check if the numbers in the profit ratio were created correctly
filtered_df.head()

Category             City        Country    Customer Name  \
4   Office Supplies  Fort Lauderdale  United States   Sean O'Donnell   
8   Office Supplies      Los Angeles  United States  Brosina Hoffman   
9   Office Supplies      Los Angeles  United States  Brosina Hoffman   
12  Office Supplies          Concord  United States     Andrew Allen   
13  Office Supplies          Seattle  United States     Irene Maddox   

    Discount  Order Date        Order ID Manufacturer  \
4        0.2  2016-10-11  US-2016-108966        Eldon   
8        0.2  2015-06-09  CA-2015-115812          DXL   
9        0.0  2015-06-09  CA-2015-115812       Belkin   
12       0.2  2018-04-15  CA-2018-114412        Xerox   
13       0.2  2017-12-05  CA-2017-161389     Fellowes   

                                         Product Name    Profit  Quantity  \
4                      Eldon Fold 'N Roll Cart System    2.5164         2   
8   DXL Angle-View Binders with Locking Rings by S...    5.7825         3   
9                    Belkin F5C206VTEL 6 Outlet Surge   34.4700         5   
12                                         Xerox 1967    5.4432         3   
13        Fellowes PB200 Plastic Comb Binding Machine  132.5922         3   

   Region    Sales   Segment   Ship Date       Ship Mode           State  \
4   South   22.368  Consumer  2016-10-18  Standard Class         Florida   
8    West   18.504  Consumer  2015-06-14  Standard Class      California   
9    West  114.900  Consumer  2015-06-14  Standard Class      California   
12  South   15.552  Consumer  2018-04-20  Standard Class  North Carolina   
13   West  407.976  Consumer  2017-12-10  Standard Class      Washington   

   Sub-Category    type purchase  profit ratio  
4       Storage     Low purchase      0.832820  
8       Binders     Low purchase     -1.159253  
9    Appliances  Medium purchase     -1.294574  
12        Paper     Low purchase      9.019938  
13      Binders     Big purchase     26.197523

In [39]:
# Now let us just filter only those rows for which the profit ratio is bigger than 8%.
office_supplies = filtered_df[filtered_df['profit ratio']>8]

In [40]:
# We can now verify that the new subset named office_supplies only has those subcategories where the profit ratio is bigger than 8% 
office_supplies.groupby(['Sub-Category','type purchase']).agg({'profit ratio': ['mean']})

profit ratio
                                     mean
Sub-Category type purchase               
Appliances   Big purchase       18.667841
Binders      Big purchase       26.197523
Paper        Big purchase        8.300845
             Low purchase        9.019938
             Medium purchase    15.520242
Storage      Big purchase       17.519520

In [41]:
# Create a data frame with just the states and the values for rating we want plot for the copiers subcategory
data_map2 = office_supplies[['State','Profit']]
data_map2.head()

State    Profit
12  North Carolina    5.4432
13      Washington  132.5922
16       Wisconsin   13.3176
34           Texas    9.9468
56        New York   15.5520

In [42]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_map2,
    columns = ['State','Profit'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'RdPu', fill_opacity=0.7, line_opacity=0.1,
    legend_name = "Profit").add_to(map)
folium.LayerControl().add_to(map)

map

From the map we can see that they are seven key regions: Nevada, Missouri, Vermont, Massachusetts, Kentuky, Delaware and Louisiana.  

#### What can we answer so far from previous Questions?
<br>-Which categories should SuperStore consider discontinue its selling?

Probably for the subcategories of medium and low purchases from binders, appliances and storage. As well, other subcategories worth of questioning are fasteners and supplies. However, this needs to be revised as these subcategories might play a key role on offering customers a all-in-one place solution where they can find anything they need and customers avoid switching to the competition.

### Category of Furniture

The category of furniture is the least profitable one, having big sales contirbute nearly 5,5% of profit and small and medium purchases only contributing to less than 1%. <br>
Having said, let us just explore at the big purchases. 

In [43]:
df_furniture = df_ss[(df_ss['Category']=='Furniture') & (df_ss['type purchase']=='Big purchase')]

In [44]:
# To discard outliers let us group the technology category by type of purchase and subcategory. 
df_furniture.groupby(['Sub-Category','type purchase']).agg({'Sales': ['sum','mean', 'min', 'max', 'count', 'median']})

Sales                                       \
                                    sum        mean      min       max count   
Sub-Category type purchase                                                     
Bookcases    Big purchase   103797.1265  687.398189  231.920  4404.900   151   
Chairs       Big purchase   299721.8570  725.718782  230.280  4416.174   413   
Furnishings  Big purchase    44044.6260  444.895212  238.152  1336.440    99   
Tables       Big purchase   196756.7850  799.824329  233.860  4297.644   246   

                                      
                              median  
Sub-Category type purchase            
Bookcases    Big purchase   424.9575  
Chairs       Big purchase   544.0080  
Furnishings  Big purchase   341.9600  
Tables       Big purchase   608.7500

In [45]:
df_furniture.groupby(['Sub-Category']).agg({'Profit': ['sum', 'count']}).apply(lambda x: 100 * x / float(x.sum()))

Profit           
                     sum      count
Sub-Category                       
Bookcases      -4.081902  16.611661
Chairs        166.103015  45.434543
Furnishings    33.558920  10.891089
Tables        -95.580033  27.062706

In here we can clearly see how Super Store is loosing money selling Tables and bookcases.
<br> For the visualizations will only use Chairs.

In [46]:
df_chairs = df_furniture[df_furniture['Sub-Category']=='Chairs']

In [47]:
# Create a data frame with just the states and the values for rating we want plot for the copiers subcategory
data_map3 = office_supplies[['State','Profit']]
data_map3.head()

State    Profit
12  North Carolina    5.4432
13      Washington  132.5922
16       Wisconsin   13.3176
34           Texas    9.9468
56        New York   15.5520

In [48]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_map3,
    columns = ['State','Profit'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'RdPu', fill_opacity=0.7, line_opacity=0.1,
    legend_name = "Profit").add_to(map)
folium.LayerControl().add_to(map)

map

Overall, these category is the least profitable of all. However, for Chairs which seems to be most attractive subcategory,  North Carolina and Illinois make negative profit. Moreover, more indepth understanding of what is causing these negative results that it isn't explained by the variables provided in the dataset is advisable.    

## 3. New Questions
How does the shipment mode affect profitability? 

Are highly discounted items the least profitable? 